# Downloading weather data
We will extracting information about local weather from National Weather Service website. The first step is to find the page we want to scrape. 
We'll extract weather information about downtown San Francisco from this page.
[Weather Forecast](https://www.weather.gov/)


## Weather Forecast Page Image
![Weather Forecast Page](https://www.dataquest.io/blog/content/images/extended_forecast.png)
As you can see from the image, the page has information about the extended forecast for the next week, including time of day, temperature, and a brief description of the conditions.

## Exploring page structure with Chrome DevTools
The first thing we'll need to do is inspect the page using Chrome Devtools. 
You can start the developer tools in Chrome by clicking `View -> Developer -> Developer Tools`. You should end up with a panel at the bottom of the browser like what you see below. Make sure the Elements panel is highlighted.
By right clicking on the page near where it says "Extended Forecast", then clicking "Inspect", we'll open up the tag that contains the text "Extended Forecast" in the elements panel:
![Extended Forecast](https://www.dataquest.io/blog/content/images/div.png)

# Extracting Weather Page
If you click around on the console, and explore the div, you'll discover that each forecast item (like "Tonight", "Thursday", and "Thursday Night") is contained in a div with the class tombstone-container.

We now know enough to download the page and start parsing it. In the below code, we:

- Download the web page containing the forecast.
- Create a BeautifulSoup class to parse the page.
- Find the div with id seven-day-forecast, and assign to seven_day
- Inside seven_day, find each individual forecast item.
- Extract and print the first forecast item.

In [3]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')

seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")

tonight = forecast_items[0]

print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  This
  <br/>
  Afternoon
 </p>
 <p>
  <img alt="This Afternoon: Sunny, with a high near 54. West wind around 10 mph. " class="forecast-icon" src="newimages/medium/skc.png" title="This Afternoon: Sunny, with a high near 54. West wind around 10 mph. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 54 °F
 </p>
</div>


## Extracting information from the page
As you can see, inside the forecast item tonight is all the information we want. There are 4 pieces of information we can extract:

- Name of forecast item — in this case, Tonight.
- Description of conditions — this is stored in the title property of img.
- A short description of the conditions — in this case, Mostly Clear.
- The temperature low — in this case, 49 degrees.

We'll extract name of forecast item, short description, and temperature first, since they're all similar:

In [4]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

ThisAfternoon
Sunny
High: 54 °F


Now, we can extract the title attribute from the img tag. To do this, we just treat the BeautifulSoup object like a dictionary, and pass in the attribute we want as a key:

In [5]:
img = tonight.find("img")
desc = img['title']

print(desc)

This Afternoon: Sunny, with a high near 54. West wind around 10 mph. 


## Extracting all information from page
Now that we know how to extract each individual piece of information, we can combine our knowledge with css selectors and list comprehensions to extract everything at once.

In the below code, we:

- Select all items with the class **`period-name`** inside an item with the class **`tombstone-container`** in **seven_day**.
- Use a list comprehension to call the get_text method on each BeautifulSoup object.

In [6]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['ThisAfternoon',
 'Tonight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight',
 'Tuesday']

We got each of the period names, in order. We can apply the same technique to get the other 3 fields:

In [7]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Sunny', 'Mostly Clear', 'Partly Sunny', 'Mostly Cloudy', 'Mostly Cloudy', 'Chance Rain', 'Chance Rain', 'Rain Likely', 'Rain Likely']
['High: 54 °F', 'Low: 45 °F', 'High: 55 °F', 'Low: 48 °F', 'High: 56 °F', 'Low: 50 °F', 'High: 58 °F', 'Low: 52 °F', 'High: 59 °F']
['This Afternoon: Sunny, with a high near 54. West wind around 10 mph. ', 'Tonight: Mostly clear, with a low around 45. West wind around 7 mph. ', 'Saturday: Partly sunny, with a high near 55. West wind 6 to 11 mph. ', 'Saturday Night: Mostly cloudy, with a low around 48. West wind 5 to 8 mph becoming light and variable. ', 'Sunday: Mostly cloudy, with a high near 56. South southeast wind 7 to 11 mph becoming west southwest in the afternoon. ', 'Sunday Night: A 30 percent chance of rain.  Cloudy, with a low around 50. New precipitation amounts of less than a tenth of an inch possible. ', 'Monday: A 50 percent chance of rain.  Cloudy, with a high near 58.', 'Monday Night: Rain likely.  Cloudy, with a low around 52.', 'Tuesd

## Combining our data into a Pandas Dataframe
We can now combine the data into a Pandas DataFrame and analyze it. A DataFrame is an object that can store tabular data, making data analysis easy. 
In order to do this, we'll call the DataFrame class, and pass in each list of items that we have. We pass them in as part of a dictionary. Each dictionary key will become a column in the DataFrame, and each list will become the values in the column:

In [8]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,period,short_desc,temp,desc
0,ThisAfternoon,Sunny,High: 54 °F,"This Afternoon: Sunny, with a high near 54. We..."
1,Tonight,Mostly Clear,Low: 45 °F,"Tonight: Mostly clear, with a low around 45. W..."
2,Saturday,Partly Sunny,High: 55 °F,"Saturday: Partly sunny, with a high near 55. W..."
3,SaturdayNight,Mostly Cloudy,Low: 48 °F,"Saturday Night: Mostly cloudy, with a low arou..."
4,Sunday,Mostly Cloudy,High: 56 °F,"Sunday: Mostly cloudy, with a high near 56. So..."
5,SundayNight,Chance Rain,Low: 50 °F,Sunday Night: A 30 percent chance of rain. Cl...
6,Monday,Chance Rain,High: 58 °F,"Monday: A 50 percent chance of rain. Cloudy, ..."
7,MondayNight,Rain Likely,Low: 52 °F,"Monday Night: Rain likely. Cloudy, with a low..."
8,Tuesday,Rain Likely,High: 59 °F,"Tuesday: Rain likely. Cloudy, with a high nea..."


We can now do some analysis on the data. For example, we can use a regular expression and the Series.str.extract method to pull out the numeric temperature values:

In [9]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    54
1    45
2    55
3    48
4    56
5    50
6    58
7    52
8    59
Name: temp_num, dtype: object

We could then find the mean of all the high and low temperatures:

In [10]:
weather["temp_num"].mean()

53.0

We could also only select the rows that happen at night:

In [11]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [12]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
1,Tonight,Mostly Clear,Low: 45 °F,"Tonight: Mostly clear, with a low around 45. W...",45,True
3,SaturdayNight,Mostly Cloudy,Low: 48 °F,"Saturday Night: Mostly cloudy, with a low arou...",48,True
5,SundayNight,Chance Rain,Low: 50 °F,Sunday Night: A 30 percent chance of rain. Cl...,50,True
7,MondayNight,Rain Likely,Low: 52 °F,"Monday Night: Rain likely. Cloudy, with a low...",52,True


## Next Steps
You should now have a good understanding of how to scrape web pages and extract data. A good next step would be to pick a site and try some web scraping on your own. Some good examples of data to scrape are:

- News articles
- Sports scores
- Weather forecasts
- Stock prices
- Online retailer prices

# Credits
## Dataquest
[Python Web Scraping Tutorial using BeautifulSoup](https://www.dataquest.io/blog/web-scraping-tutorial-python/)